#  Final Project: Etsy Marketplace Exploration

## Data Aquisition

### Get all Categories and Sub Categories

In [1]:
# import tools
from lxml import html
import xml.etree.ElementTree as ET
import time
import requests
import lxml
import pandas as pd
import csv
import time
import re
from IPython import embed

#### Read in etsy_category_pages.csv

In [ ]:
# read in the list of etsy page categories
etsy_pages = pd.read_csv("etsy_category_pages.csv")

In [ ]:
page_cat = etsy_pages['page'].tolist()

In [ ]:
# generate list of urls for each category
urls = []
start = "https://www.etsy.com/c/"

for x in page_cat:
    urls.append(start + x)

In [ ]:
# Find all html elements which contain the links to the subcategory pages
sub_elements = []

for url in urls:
    #send the request
    content = requests.get(url)
    #save the page source code to a string called content_string
    content_string = content.text.encode("utf-8")
    #pass the page source to our html parse
    doc = lxml.html.document_fromstring(content_string)
    # Find html element containing each subcategory
    element = doc.xpath("//li[@class='pb-xs-1 pl-xs-0']")
    sub_elements.append(element)

In [ ]:
# Get urls for each of the subcategories and store them in a list
sub_links = []

for element in sub_elements:
    for x in element:
        child = x.getchildren()
        new = child[0].attrib['href']
        sub_links.append(new)

In [ ]:
# Save subcategory links in a csv file
with open('subcategory_links.csv', 'w') as csvfile:
    fieldnames=['link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # iterate through category and subcategory
    for item in sub_links:
        writer.writerow({'link': item})

#### Write subcategory_links.csv

In [ ]:
# open csv subcategory link file
# create new pandas dataframe called catdf

newdata = pd.read_csv("subcategory_links.csv",sep="/",names=('0','1','2','3','top_category','link_address'))
del(newdata['0'],newdata['1'],newdata['2'],newdata['3'])
catdf = newdata.dropna()

In [ ]:
# extract subcatgory name from link
def clean_links(text):
    return text.split("?")[0]

catdf['sub_category'] = catdf['link_address'].apply(clean_links)

In [ ]:
# Function to get urls for pages 2-250 in each subcategory
# https://www.etsy.com/c/jewelry/bracelets?page=24
def get_page2(row):
    urls = []
    start = "https://www.etsy.com/c/"
    for i in range(2,250):
        urls.append(start + row['top_category'] + "/" + row['sub_category'] + "?page=" + str(i))
    return ",".join(urls)

In [ ]:
# Apply function and story multiple page urls in column called page_nums
catdf['page_nums'] = catdf.apply(get_page2, axis=1)

In [ ]:
sub_links = pd.read_csv("subcategory_links.csv")

In [ ]:
# join dataframes
catdf = catdf.reset_index()
catdf = catdf.join(sub_links)
del catdf['index']
del catdf['link_address']

catdf.head(5)

### Get all shop names for each subcategory

In [ ]:
# finds all shop names from html source string and appends to list
# returns string
def get_shop_names(some_link):
    time.sleep(2)
    content = some_link
    source_code = requests.get(content)
    source_string = source_code.text.encode("utf-8")
    j = []
    doc = lxml.html.document_fromstring(source_string)
    tags = doc.xpath("//div[@class='card-meta-row-item text-truncate overflow-hidden card-shop-name']")
    for tag in tags:
        j.append(tag.text)
    return ",".join(j)

### should change the join character to | in case shop names contain a comma

In [ ]:
new_column = sub_links['link'].apply(get_shop_names)

In [ ]:
catdf['raw_shops'] = new_column

In [4]:
# turn string into list of strings
def link_list(a_string):
    foo = a_string.split(",")
    return foo

### should change the join character to | in case shop names contain a comma

In [ ]:
catdf['more_shops'] = ""

In [ ]:
catdf['even_more_shops'] = ""

In [ ]:
# run get_shop_names on page_nums
# j is length of catdf
# i is page of results you want to see (up to 250)

for j in range(0,161):
    trial = link_list(catdf['page_nums'][j])
    k = []
    for i in range(20,25):
        one = get_shop_names(trial[i])
        k.append(one)
    catdf['even_more_shops'][j] = ",".join(k)
    
### should change the join character to | in case shop names contain a comma

In [ ]:
catdf.to_csv(path_or_buf="shop_names")

#### Write catdf = csv called 'shop_names'

In [ ]:
# takes raw string containing shop names
#returns cleaned with shop name only separated by comas
def clean_name(messy_name):
    new_list = []
    for name in messy_name:
        finder = re.compile("\n" + "(.*?)" + "\n", re.IGNORECASE)
        matches = finder.findall(name)
        no_replace = re.compile("\s+")
        p = no_replace.sub("", matches[0])
        new_list.append(p)
    return ",".join(new_list)

### should change the join character to | in case shop names contain a comma

In [ ]:
catdf['all_shops'] = ""

In [ ]:
catdf['all2_shops'] = ""

In [ ]:
catdf['all3_shops'] = ""

In [ ]:
# clean more_shops and store names in all_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['more_shops'][i])
    catdf['all_shops'][i] = clean_name(bar)

In [ ]:
# clean even_more_shops and store names in all_shops2
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['even_more_shops'][i])
    catdf['all2_shops'][i] = clean_name(bar)

In [ ]:
# clean raw_shops and store names in all_shops3
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['raw_shops'][i])
    catdf['all3_shops'][i] = clean_name(bar)

In [ ]:
def appendcols(row):
    s = row['all_shops'] + row['all_shops'] + row['all_shops']
    return s

In [ ]:
catdf['shop_names'] = catdf.apply(appendcols, axis=1)

In [ ]:
final_catdf = catdf[['top_category','sub_category','shop_names']]

In [ ]:
for i in range(120,161):
    print i
    print type(final_catdf['shop_names'][i])

In [ ]:
final_catdf.to_csv(path_or_buf="shop_names")

#### Write final_catdf = csv called 'shop_names'

In [8]:
###Change file name so previous file does not get overwritten

In [2]:
final_catdf

,Unnamed: 0,top_category,sub_category,shop_names
0,0,accessories,hair-accessories,"TheIvoryWillowCo,ChicBlue,TheBridesStore,Lovel..."
1,1,accessories,hats-and-caps,"RockyMountainHatCo,RambleBabyBoutique,Rawsells..."
2,2,accessories,baby-accessories,"SkylarnMe,SweetAndStitched,LacoHandMade,Simply..."
3,3,accessories,scarves-and-wraps,"MiracleShine,ScarfClub,Zojanka,LTCoutureAtelie..."
4,4,accessories,keychains-and-lanyards,"artbymorgie,accessoriesbyjamiep,NiOuiNinoucheG..."
5,5,accessories,suit-and-tie-accessories,"BridalCloset,LoveVanillaDew,ElegantCuff,TieObs..."
6,6,accessories,patches-and-pins,"dannybrito,ilikeCATSshop,DistinctDesignsUnltd,..."
7,7,accessories,belts-and-suspenders,"RhythmicMetal,ISHAOR,armoniia,TheLeatherBoy,Th..."
8,8,accessories,costume-accessories,"Merimask,superheroforparty,Customownnecklace,K..."
9,9,accessories,sunglasses-and-eyewear,"SunglassMuseum,retroandme,Propwood,CarvedAcces..."


### Random Samples of Shops

Sample set of data will be 10 random shops from each category. This way each category is equally represented, but fewer review pages will need to be pulled.


In [ ]:
# this will pull random sample from all shops and store them in sample shop names

for i in range(0,161): #this is the length of final_catdf
    namelist = catdf['shop_names'][i].split(",")
    sample = np.random.choice(namelist, 10, replace = False)
    newlist = list(sample)
    output = ",".join(newlist)
    catdf['sample_shop_names'][i] = output
    
### should change the join character to | in case shop names contain a comma

In [ ]:
completelist = []
for i in range(0,161):
    a = (catdf['sample_shop_names'][i].split(","))
    for b in a:
        completelist.append(b)
        
### should change the join character to | in case shop names contain a comma

In [ ]:
sampledf = pd.DataFrame(data = completelist, columns = ["shops"])

In [ ]:
# write sample shop list to csv

sampledf.to_csv(path_or_buf="sample_shop_names")

### Get reviews for all shops in random sample

In [ ]:
# drop duplicate shop names
# add new columns

In [2]:
sample_shops = pd.read_csv("sample_shop_names.csv")

In [72]:
del sample_shops['Unnamed: 0']

In [73]:
sample_shops.drop_duplicates(inplace = True)

In [74]:
sample_shops = sample_shops.reset_index()

In [44]:
# get dates, ratings and details for each shop
# run 0,1485 for complete set

for i in range(0,1485): # index of shop
    current_page = ''
    for page_number in range(1,10): # review page number, starts with 1 not 0
        
        # send request to url
        time.sleep(.75)
        base_url = "https://www.etsy.com/shop/SHOP_NAME/reviews?ref=pagination&page=PAGE_NUMBER"
        url = base_url.replace('PAGE_NUMBER', str(page_number))
        url = url.replace('SHOP_NAME', sample_shops['shops'][i])
        source_code = requests.get(url)
        if 'a-review-too-far' in source_code.text:
            print "out of pages"
            break
            
        source_string = source_code.text.encode("utf-8")
        doc = lxml.html.document_fromstring(source_string)
        
        #find xpath with item details
        n_elements = []
        m_elements = []
        list_details = []
        tags3 = doc.xpath("//div[@class='flag-body hide-xs hide-sm']")
        for tag in tags3:
            n_elements.append(tag.getchildren())
        for n in n_elements:
            for x in n:
                m_elements.append(x.getchildren())
        for m in m_elements:
            for x in m:
                list_details.append(x.text)
        column_name_item = 'item_details' + str(page_number)
        if column_name_item not in sample_shops:
            sample_shops[column_name_item] = ""
        sample_shops[column_name_item][i] = "|".join(list_details)
    
        #find xpath with review dates
        p_elements = []
        a_elements = []
        list_dates = []
        tags = doc.xpath("//div[@class='mt-xs-2 mb-xs-2']")
        for tag in tags:
            p_elements.append(tag.getchildren())
        for p in p_elements:
            for x in p:
                a_elements.append(x.getchildren())
        for element in a_elements:
            for x in element:
                list_dates.append(x.tail)
        column_name_date = 'review_dates' + str(page_number)
        if column_name_date not in sample_shops:
            sample_shops[column_name_date] = ""
        sample_shops[column_name_date][i] = "|".join(list_dates)

        # find xpath with review ratings
        span_elements = []
        tags2 = doc.xpath("//span[@class='screen-reader-only']") 
        for tag in tags2:
            span_elements.append(tag.text)
        column_name_ratings = 'ratings' + str(page_number)
        if column_name_ratings not in sample_shops:
            sample_shops[column_name_ratings] = ""
        sample_shops[column_name_ratings][i] = "|".join(span_elements)
    
    
    print i

out of pages
1102


C:\Users\jishii\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jishii\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jishii\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1103
1104
1105
1106
out of pages
1107
1108
out of pages
1109
1110
1111
1112
1113
1114
1115
1116
1117
out of pages
1118
1119
1120
1121
1122
out of pages
1123
1124
out of pages
1125
1126
out of pages
1127
1128
out of pages
1129
1130
out of pages
1131
1132
out of pages
1133
1134
1135
out of pages
1136
1137
1138
out of pages
1139
1140
1141
1142
1143
1144
out of pages
1145
out of pages
1146
1147
out of pages
1148
out of pages
1149
1150
out of pages
1151
1152
out of pages
1153
1154
1155
1156
1157
1158
1159
out of pages
1160
1161
1162
1163
1164
1165
out of pages
1166
out of pages
1167
1168
out of pages
1169
out of pages
1170
1171
1172
1173
1174
out of pages
1175
out of pages
1176
1177
1178
1179
out of pages
1180
out of pages
1181
1182
out of pages
1183
out of pages
1184
out of pages
1185
1186
1187
out of pages
1188
out of pages
1189
1190
1191
out of pages
1192
1193
1194
out of pages
1195
1196
out of pages
1197
1198
1199
1200
1201
1202
1203
1204
out of pages
1205
1206
1207
1208
1209
1210
1211


In [41]:
sample_shops[1100:1105]

,Unnamed: 0,index,shops,item_details1,review_dates1,ratings1,item_details2,review_dates2,ratings2,item_details3,...,ratings6,item_details7,review_dates7,ratings7,item_details8,review_dates8,ratings8,item_details9,review_dates9,ratings9
1100,1100,1193,SpaGoddess,Rose Crystal Moisturizing Organic Rosewater Fa...,"on Aug 22, 2016\n | on Aug...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,SALE - Kissed Organic Facial Cleanser - Gentl...,"on Jul 31, 2016\n | on Jul...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Illuminesce Glow + Repair Serum. Organic Facia...,...,Search|5 out of 5 stars|5 out of 5 stars|5 out...,Super-Detox Deep Pore Facial Cleanser with Act...,"on Mar 10, 2016\n | on Mar...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Grace Antioxidant Organic Facial Toner -- RESE...,"on Feb 4, 2016\n | on Feb ...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Blemish Soother - Organic Acne Spot Treatment ...,"on Jan 9, 2016\n | on Jan ...",Search|5 out of 5 stars|5 out of 5 stars|5 out...
1101,1101,1194,SeasideSoapKitchen,Pineapple Soap - Pineapple Cilantro Soap - Gly...,"on Aug 22, 2016\n | on Aug...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,"Reserved for Melissa|Oatmeal, Milk & Honey - g...","on Aug 9, 2016\n | on Aug ...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Sweet Orange Soap - Glycerin Soap - Citrus Soa...,...,Search|5 out of 5 stars|5 out of 5 stars|5 out...,Reserved for Kristie|Starfish Soap - Set of 5 ...,"on Jun 2, 2016\n | on Jun ...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Reserved for Melissa|Eucalyptus Mint - glyceri...,"on May 25, 2016\n | on May...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Fish Soap - Fish in a Bag Soap Set of 10 - Fis...,"on May 11, 2016\n | on May...",Search|5 out of 5 stars|5 out of 5 stars|5 out...
1102,1102,1195,AmbiguousYou,Ultimate Lip Set- All Natural & Vegan|Sugar Bo...,"on Aug 5, 2016\n | on Aug ...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Eyeshadow Set of 3 Different Mineral Eyeshadow...,"on Nov 23, 2015\n | on Nov...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Rosy Cheeks Matte- Mineral Blush- All Natural/...,...,,,,,,,,,,
1103,1103,1196,SandyLandStudio,"Vanilla Soy Candle, Herbal Bath Tea and Bath S...","on Aug 14, 2016\n | on Aug...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,4 Oz Soy Candle Choose your Scent Eco Wick Hig...,"on Jun 30, 2016\n | on Jun...",Search|5 out of 5 stars|5 out of 5 stars|3 out...,Reserved for Janene Luna Moth Original Painti...,...,Search|5 out of 5 stars|5 out of 5 stars|5 out...,Tangerine European Spa Salt and Dead Sea Bath ...,"on Dec 30, 2015\n | on Dec...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Custom Hand Painted Ceramic Christmas Ornament...,"on Dec 14, 2015\n | on Nov...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,4 Oz Soy Candle Choose your Scent Eco Wick Hig...,"on Sep 9, 2015\n | on Aug ...",Search|5 out of 5 stars|5 out of 5 stars|5 out...
1104,1104,1197,GlowBodyandSoul,Coconut Banana Smoothie Body Butter|Gumdrops a...,"on Aug 23, 2016\n | on Aug...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Warm Vanilla Sugar Emulsified Sugar Scrub 6oz....,"on Aug 14, 2016\n | on Aug...",Search|5 out of 5 stars|5 out of 5 stars|5 out...,Lip Scrub Choose Your Flavor Organic Cane Suga...,...,Search|5 out of 5 stars|5 out of 5 stars|5 out...,,,,,,,,,


In [45]:
### check if string has any unicode characters

for i in range(1102,1485): # index of shops
    for j in range(6,10): # review page number
        if type(sample_shops['item_details'+ str(j)][i]) == str:
            #print "a" + str(i) + str(j)
        elif type(sample_shops['item_details'+ str(j)][i]) == float:
            #print "f" + str(i) + str(j)
        elif type(sample_shops['item_details'+str(j)][i]) != (str or float):
            s = sample_shops['item_details'+ str(j)][i].encode("utf-8")
            print "unicode" + str(i) + str(j)
            sample_shops['item_details' + str(j)][i] = s

a11026
a11027
a11028
a11029
a11036
a11037
a11038
a11039
a11046
a11047
a11048
a11049
a11056
a11057
a11058
a11059
a11066
a11067
a11068
a11069
a11076
a11077
a11078
a11079
a11086
a11087
a11088
a11089
a11096
a11097
a11098
a11099
a11106
a11107
a11108
a11109
a11116
a11117
a11118
a11119
a11126
unicode11127
a11128
unicode11129
a11136
a11137
a11138
a11139
a11146
a11147
a11148
a11149
a11156
a11157
a11158
a11159
a11166
a11167
a11168
a11169
a11176
a11177
a11178
a11179
a11186
a11187
a11188
a11189
a11196
a11197
a11198
a11199
a11206
a11207
a11208
a11209
a11216
a11217
a11218
a11219
a11226
a11227
a11228
a11229
a11236
a11237
a11238
a11239
a11246
a11247
a11248
a11249
a11256
a11257
a11258
a11259
unicode11266
unicode11267
unicode11268
a11269
a11276
a11277
a11278
a11279
a11286
a11287
a11288
a11289
a11296
a11297
a11298
a11299
unicode11306
a11307
unicode11308
unicode11309


C:\Users\jishii\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


a11316
a11317
a11318
a11319
a11326
a11327
a11328
a11329
a11336
a11337
a11338
a11339
a11346
a11347
a11348
a11349
a11356
a11357
a11358
a11359
a11366
a11367
a11368
a11369
a11376
a11377
a11378
a11379
a11386
a11387
a11388
a11389
a11396
a11397
a11398
a11399
a11406
a11407
a11408
a11409
a11416
a11417
a11418
a11419
a11426
a11427
a11428
a11429
a11436
unicode11437
unicode11438
unicode11439
a11446
a11447
a11448
a11449
a11456
a11457
a11458
a11459
a11466
a11467
a11468
a11469
a11476
a11477
a11478
a11479
a11486
a11487
a11488
a11489
a11496
a11497
a11498
a11499
a11506
a11507
a11508
a11509
a11516
a11517
a11518
a11519
a11526
unicode11527
a11528
a11529
a11536
a11537
a11538
a11539
a11546
a11547
a11548
a11549
a11556
a11557
a11558
a11559
unicode11566
unicode11567
a11568
unicode11569
a11576
a11577
a11578
a11579
a11586
a11587
a11588
a11589
a11596
a11597
a11598
a11599
a11606
a11607
a11608
a11609
a11616
a11617
a11618
a11619
a11626
a11627
a11628
a11629
a11636
a11637
unicode11638
a11639
a11646
a11647
a11648
a11649


In [46]:
# write to csv file

sample_shops.to_csv(path_or_buf = "raw_shop_reviews.csv")

#### Write sample_shops = raw_shop_reviews.csv